# DataFrames: Groupby

This notebook uses the Pandas groupby-aggregate and groupby-apply on scalable Dask dataframes.  It will discuss both common use and best practices.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [ ]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

## Artifical dataset

We create an artificial timeseries dataset to help us work with groupby operations

In [ ]:
import dask
df = dask.datasets.timeseries()
df

This dataset is small enough to fit in the cluster's memory, so we persist it now.

You would skip this step if your dataset becomes too large to fit into memory.

In [ ]:
df = df.persist()

## Groupby Aggregations

Dask dataframes implement a commonly used subset of the Pandas groupby API (see [Pandas Groupby Documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

We start with groupby aggregations.  These are generally fairly efficient, assuming that the number of groups is small (less than a million).

In [ ]:
df.groupby('name').x.mean().compute()

Performance will depend on the aggregation you do (mean vs std), the key on which you group (name vs id), and the number of total groups

In [ ]:
%time _ = df.groupby('id').x.mean().compute()

In [ ]:
%time _ = df.groupby('name').x.mean().compute()

In [ ]:
%time df.groupby('name').agg({'x': ['mean', 'std'], 'y': ['mean', 'count']}).compute().head()

This is the same as with Pandas.  Generally speaking, Dask.dataframe groupby-aggregations are roughly same performance as Pandas groupby-aggregations, just more scalable.

You can read more about Pandas' common aggregations in [the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html#aggregation).



## Custom Aggregations

Dask dataframe Aggregate is available for custom aggregations (See [Dask dataframe Aggregate Documentation](https://docs.dask.org/en/latest/dataframe-groupby.html#aggregate))

## Many groups

By default groupby-aggregations (like groupby-mean or groupby-sum) return the result as a single-partition Dask dataframe.  Their results are *usually* quite small, so this is *usually* a good choice.

However, sometimes people want to do groupby aggregations on *many* groups (millions or more).  In these cases the full result may not fit into a single Pandas dataframe output, and you may need to split your output into multiple partitions.  You can control this with the `split_out=` parameter

In [ ]:
# Computational graph of a single output aggregation (for a small number of groups, like 1000)
df.groupby('name').x.mean().visualize(node_attr={'penwidth': '6'})

In [ ]:
# Computational graph of an aggregation to four outputs (for a larger number of groups, like 1000000)
df.groupby('id').x.mean(split_out=4).visualize(node_attr={'penwidth': '6'})

## Groupby Apply

Groupby-aggregations are generally quite fast because they can be broken down easily into well known operations.  The data doesn't have to move around too much and we can just pass around small intermediate values across the network.

For some operations however the function to be applied requires *all* data from a given group (like every record of someone named "Alice").  This will force a great deal of communication and be more expensive, but is still possible with the Groupby-apply method.  This should be avoided if a groupby-aggregation works.

In the following example we train a simple Scikit-Learn machine learning model on every person's name.

In [ ]:
from sklearn.linear_model import LinearRegression

def train(partition):
    est = LinearRegression()
    est.fit(partition[['x', 'id']].values, partition.y.values)
    return est

In [ ]:
%time df.groupby('name').apply(train, meta=object).compute().sort_index()